In [1]:
import dataloader
import pandas as pd
import utils
from model import generate_model
import os
import torch
import torch.nn as nn

from optimizer import Adam,SGD

from train_wrapper import train_epoch

In [2]:
config = utils.load_config()

batch_size = config['dataloader']['batch_size']
num_workers = config['dataloader']['num_workers']
pin_memory = config['dataloader']['pin_memory'] == 1 
gpu_parallel = config['gpus']
learning_rate = config['train']['learning_rate']
lr_steps = config['train']['lr_steps']
epochs = config['train']['epochs']

In [3]:
#데이터셋 분리(Train, validation, test)
df_dataset = pd.read_csv(config['PATH_DATASET_CSV'])
df_dataset = df_dataset.dropna().reset_index(drop=True)
df_oasis = df_dataset[df_dataset['source'] == 'OASIS-3']
df_adni = df_dataset[df_dataset['source'] == 'ADNI']
X_train,X_val,y_train,y_val = dataloader.dataset_split(df_oasis,test_size=0.2,shuffle=True,grp=None,seed=1004)
X_test = df_adni.drop(labels='group_maxinc',axis=1)
y_test = df_adni['group_maxinc']

d:\MyGit\BrainMR_MCI\dataloader.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dataset['grp'] = (df_dataset['source'].str.replace('OASIS-3','1').str.replace('ADNI','2').apply(pd.to_numeric)*1000


In [4]:
traindata=dataloader.MRIDataset(X_train,y_train)
valdata=dataloader.MRIDataset(X_val,y_val)
testdata=dataloader.MRIDataset(X_test,y_test)

from torch.utils.data import DataLoader
train_dataloader = DataLoader(traindata, batch_size=batch_size, shuffle=True
                              ,num_workers=num_workers,pin_memory = pin_memory)
val_dataloader  = DataLoader(valdata, batch_size=batch_size, shuffle=False
                              ,num_workers=num_workers,pin_memory = pin_memory)
test_dataloader  = DataLoader(testdata, batch_size=1, shuffle=False)

print('train_dataloader : ',len(train_dataloader.dataset))
print('val_dataloader : ',len(val_dataloader.dataset))
print('test_dataloader : ',len(test_dataloader.dataset))

train_dataloader :  659
val_dataloader :  165
test_dataloader :  166


d:\MyGit\BrainMR_MCI\pymain\lib\site-packages\torch\utils\data\dataloader.py:554: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 4 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

model_name = config['model']['model_name']
model_depth = config['model']['model_depth']

model, _ = generate_model(model_name=model_name,model_depth = model_depth,n_classes=3,resnet_shortcut='B')
model.to(device)

if len(gpu_parallel) > 1 and torch.cuda.is_available():
    model = nn.DataParallel(model, device_ids = gpu_parallel)
    model.to(device)

optimizer = Adam(model, learning_rate= learning_rate)
criterion_clf = nn.CrossEntropyLoss().to(device)

cpu


d:\MyGit\BrainMR_MCI\models\resnet.py:140: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  m.weight = nn.init.kaiming_normal(m.weight, mode='fan_out')


In [6]:
train_epoch(device,train_dataloader,val_dataloader,model,criterion_clf,optimizer,config
            ,epoch=epochs,learning_rate= learning_rate,lr_steps=lr_steps)

train at epoch 0
